# Social Network Analysis: Six Degrees of Separation

This notebook implements a comprehensive analysis of the soc-Pokec dataset to empirically test the "six degrees of separation" theory.

## Project Overview
- **Dataset**: soc-Pokec (Slovak social network) - 1.6M nodes, 30M edges
- **Goal**: Compute average shortest path lengths, analyze network properties, and perform community detection
- **Key Techniques**: Graph mining, centrality analysis, community detection, link prediction

## Step 1: Setup and Data Acquisition

In [ ]:
import os
import requests
import gzip
import shutil
from pathlib import Path

# Create data directory if it doesn't exist
DATA_DIR = Path('data')
DATA_DIR.mkdir(exist_ok=True)

# Dataset URLs from Stanford SNAP
URLS = {
    'edges': 'https://snap.stanford.edu/data/soc-pokec-relationships.txt.gz',
    'profiles': 'https://snap.stanford.edu/data/soc-pokec-profiles.txt.gz',
    'readme': 'https://snap.stanford.edu/data/soc-pokec-readme.txt'
}

def download_file(url, dest_path):
    """Download a file with progress indication."""
    print(f"Downloading {url}...")
    response = requests.get(url, stream=True)
    response.raise_for_status()
    
    total_size = int(response.headers.get('content-length', 0))
    downloaded = 0
    
    with open(dest_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
            downloaded += len(chunk)
            if total_size > 0:
                pct = (downloaded / total_size) * 100
                print(f"\rProgress: {pct:.1f}%", end='', flush=True)
    print(f"\nSaved to {dest_path}")

def extract_gzip(gz_path, output_path):
    """Extract a gzip file."""
    print(f"Extracting {gz_path}...")
    with gzip.open(gz_path, 'rb') as f_in:
        with open(output_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    print(f"Extracted to {output_path}")

In [ ]:
# Download and extract dataset files
files_to_download = [
    ('edges', DATA_DIR / 'soc-pokec-relationships.txt.gz', DATA_DIR / 'soc-pokec-relationships.txt'),
    ('profiles', DATA_DIR / 'soc-pokec-profiles.txt.gz', DATA_DIR / 'soc-pokec-profiles.txt'),
]

for name, gz_path, txt_path in files_to_download:
    if not txt_path.exists():
        if not gz_path.exists():
            download_file(URLS[name], gz_path)
        extract_gzip(gz_path, txt_path)
    else:
        print(f"{txt_path} already exists, skipping download.")

# Download readme
readme_path = DATA_DIR / 'soc-pokec-readme.txt'
if not readme_path.exists():
    download_file(URLS['readme'], readme_path)
else:
    print(f"{readme_path} already exists, skipping download.")

In [ ]:
# Verify downloaded files
for file in DATA_DIR.glob('*.txt'):
    line_count = sum(1 for _ in open(file, 'r', encoding='utf-8', errors='ignore'))
    size_mb = file.stat().st_size / (1024 * 1024)
    print(f"{file.name}: {line_count:,} lines, {size_mb:.2f} MB")

## Step 2: Data Loading and Preprocessing

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import psutil
import time

def get_memory_usage():
    """Return current memory usage in GB."""
    process = psutil.Process()
    return process.memory_info().rss / (1024 ** 3)

print(f"Initial memory usage: {get_memory_usage():.2f} GB")

In [ ]:
# Load edges into a NetworkX graph
print("Loading edge list...")
start_time = time.time()

edges_file = DATA_DIR / 'soc-pokec-relationships.txt'
G = nx.read_edgelist(str(edges_file), create_using=nx.DiGraph(), nodetype=int)

load_time = time.time() - start_time
print(f"Loaded in {load_time:.2f} seconds")
print(f"Directed Graph - Nodes: {G.number_of_nodes():,}, Edges: {G.number_of_edges():,}")
print(f"Memory usage: {get_memory_usage():.2f} GB")

In [ ]:
# Convert to undirected for separation calculations
print("Converting to undirected graph...")
G_undirected = G.to_undirected()
print(f"Undirected Graph - Nodes: {G_undirected.number_of_nodes():,}, Edges: {G_undirected.number_of_edges():,}")
print(f"Memory usage: {get_memory_usage():.2f} GB")

In [ ]:
# Profile columns based on soc-pokec-readme.txt
# The dataset has 59 columns
PROFILE_COLUMNS = [
    'user_id', 'public', 'completion_percentage', 'gender', 'region', 'last_login',
    'registration', 'AGE', 'body', 'I_am_working_in_field', 'spoken_languages',
    'hobbies', 'I_most_enjoy_good_food', 'pets', 'body_type', 'my_eyesight',
    'eye_color', 'hair_color', 'hair_type', 'completed_level_of_education',
    'favourite_color', 'relation_to_smoking', 'relation_to_alcohol',
    'sign_in_zodiac', 'on_pokec_i_teleasing_for', 'love_is_for_me',
    'relation_to_casual_sex', 'my_partner_should_be', 'marital_status',
    'children', 'relation_to_children', 'I_like_movies', 'I_like_watching_movie',
    'I_like_music', 'I_mostly_like_listening_to_music', 'the_idea_of_good_evening',
    'I_like_specialties_from_kitchen', 'fun', 'I_am_going_to_concerts',
    'my_active_sports', 'my_passive_sports', 'profession', 'I_like_books',
    'life_style', 'music', 'cars', 'politics', 'relationships', 'art_culture',
    'hobbies_interests', 'science_technologies', 'computers_internet',
    'education', 'sport', 'movies', 'travelling', 'health', 'companies_brands',
    'more'
]

print(f"Number of profile columns: {len(PROFILE_COLUMNS)}")

In [ ]:
# Load profiles with Pandas
print("Loading profiles...")
start_time = time.time()

profiles_file = DATA_DIR / 'soc-pokec-profiles.txt'
profiles = pd.read_csv(
    profiles_file,
    sep='\t',
    header=None,
    names=PROFILE_COLUMNS,
    dtype=str,
    low_memory=False,
    on_bad_lines='skip'
)

load_time = time.time() - start_time
print(f"Loaded {len(profiles):,} profiles in {load_time:.2f} seconds")
print(f"Memory usage: {get_memory_usage():.2f} GB")

In [ ]:
# Preview profiles
profiles.head()

In [ ]:
# Preprocess profiles
profiles['user_id'] = pd.to_numeric(profiles['user_id'], errors='coerce')
profiles = profiles.dropna(subset=['user_id'])
profiles['user_id'] = profiles['user_id'].astype(int)
profiles.set_index('user_id', inplace=True)

# Convert age to numeric
profiles['AGE'] = pd.to_numeric(profiles['AGE'], errors='coerce')

# Fill missing values
profiles.fillna('unknown', inplace=True)

print(f"Processed profiles: {len(profiles):,}")
print(f"\nAge statistics:")
print(profiles['AGE'].describe())

In [ ]:
# Attach key attributes to graph nodes (sample for efficiency)
print("Attaching profile attributes to graph nodes...")
attached_count = 0

for node in G.nodes():
    if node in profiles.index:
        try:
            G.nodes[node]['gender'] = profiles.loc[node, 'gender']
            G.nodes[node]['age'] = profiles.loc[node, 'AGE']
            G.nodes[node]['region'] = profiles.loc[node, 'region']
            attached_count += 1
        except Exception:
            pass

print(f"Attached attributes to {attached_count:,} nodes")

## Step 3: Core Analysis - Compute Degrees of Separation

In [ ]:
import random
from collections import Counter

# Get the largest connected component for meaningful path calculations
print("Finding connected components...")
components = list(nx.connected_components(G_undirected))
print(f"Number of connected components: {len(components)}")

# Get largest component
largest_cc = max(components, key=len)
print(f"Largest component size: {len(largest_cc):,} nodes ({100*len(largest_cc)/G_undirected.number_of_nodes():.2f}% of graph)")

# Create subgraph of largest component
G_lcc = G_undirected.subgraph(largest_cc).copy()

In [ ]:
# Sample and compute shortest paths
print("Computing degrees of separation (sampled)...")

NUM_SAMPLES = 10000  # Number of random pairs to sample
nodes_list = list(G_lcc.nodes())
path_lengths = []
failed_paths = 0

start_time = time.time()

for i in range(NUM_SAMPLES):
    src, tgt = random.sample(nodes_list, 2)
    try:
        length = nx.shortest_path_length(G_lcc, src, tgt)
        path_lengths.append(length)
    except nx.NetworkXNoPath:
        failed_paths += 1
    
    if (i + 1) % 1000 == 0:
        print(f"Progress: {i+1}/{NUM_SAMPLES} pairs computed")

compute_time = time.time() - start_time
print(f"\nCompleted in {compute_time:.2f} seconds")
print(f"Successful paths: {len(path_lengths):,}")
print(f"Failed paths (disconnected): {failed_paths}")

In [ ]:
# Calculate statistics
if path_lengths:
    avg_degree = np.mean(path_lengths)
    median_degree = np.median(path_lengths)
    std_degree = np.std(path_lengths)
    min_degree = min(path_lengths)
    max_degree = max(path_lengths)
    
    print("=" * 50)
    print("DEGREES OF SEPARATION ANALYSIS")
    print("=" * 50)
    print(f"Average degrees of separation: {avg_degree:.2f}")
    print(f"Median degrees of separation: {median_degree:.2f}")
    print(f"Standard deviation: {std_degree:.2f}")
    print(f"Min path length: {min_degree}")
    print(f"Max path length: {max_degree}")
    print("=" * 50)
    
    # Distribution
    print("\nPath length distribution:")
    distribution = Counter(path_lengths)
    for length in sorted(distribution.keys()):
        count = distribution[length]
        pct = 100 * count / len(path_lengths)
        bar = '█' * int(pct / 2)
        print(f"  {length}: {count:,} ({pct:.1f}%) {bar}")

In [ ]:
# Compute additional network metrics
print("Computing network metrics...")

# Degree distribution
degrees = [d for n, d in G_undirected.degree()]
avg_degree_network = np.mean(degrees)
print(f"Average node degree: {avg_degree_network:.2f}")

# Density
density = nx.density(G_undirected)
print(f"Network density: {density:.6f}")

In [ ]:
# Average clustering coefficient (sampled for large graphs)
print("Computing clustering coefficient (sampled)...")
sample_nodes = random.sample(nodes_list, min(10000, len(nodes_list)))
clustering_coeffs = nx.clustering(G_lcc, nodes=sample_nodes)
avg_clustering = np.mean(list(clustering_coeffs.values()))
print(f"Average clustering coefficient (sampled): {avg_clustering:.4f}")

In [ ]:
# Reciprocity for directed graph
reciprocity = nx.reciprocity(G)
print(f"Reciprocity (directed graph): {reciprocity:.4f}")

## Step 4: Advanced Mining Tasks

### 4.1 Community Detection

In [ ]:
import community as community_louvain

# For large graphs, use a sample for community detection
COMMUNITY_SAMPLE_SIZE = 50000

print(f"Sampling {COMMUNITY_SAMPLE_SIZE:,} nodes for community detection...")
sample_nodes_community = random.sample(nodes_list, min(COMMUNITY_SAMPLE_SIZE, len(nodes_list)))
G_sample = G_lcc.subgraph(sample_nodes_community).copy()

print(f"Sample graph - Nodes: {G_sample.number_of_nodes():,}, Edges: {G_sample.number_of_edges():,}")

In [ ]:
# Run Louvain community detection
print("Running Louvain community detection...")
start_time = time.time()

partition = community_louvain.best_partition(G_sample)

detect_time = time.time() - start_time
print(f"Community detection completed in {detect_time:.2f} seconds")

In [ ]:
# Analyze communities
community_sizes = Counter(partition.values())
num_communities = len(community_sizes)
modularity = community_louvain.modularity(partition, G_sample)

print(f"\nCommunity Detection Results:")
print(f"Number of communities: {num_communities}")
print(f"Modularity score: {modularity:.4f}")
print(f"\nTop 10 largest communities:")
for comm_id, size in community_sizes.most_common(10):
    pct = 100 * size / len(partition)
    print(f"  Community {comm_id}: {size:,} nodes ({pct:.1f}%)")

### 4.2 Centrality Measures (Identify Hubs)

In [ ]:
# Degree centrality (fast)
print("Computing degree centrality...")
degree_centrality = nx.degree_centrality(G_lcc)
top_degree = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:10]

print("\nTop 10 nodes by degree centrality:")
for node, centrality in top_degree:
    degree = G_lcc.degree(node)
    print(f"  Node {node}: centrality={centrality:.6f}, degree={degree}")

In [ ]:
# Betweenness centrality (sampled for efficiency)
print("Computing betweenness centrality (sampled)...")
start_time = time.time()

betweenness_centrality = nx.betweenness_centrality(G_sample, k=500)  # Sample k nodes

compute_time = time.time() - start_time
print(f"Computed in {compute_time:.2f} seconds")

top_betweenness = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:10]

print("\nTop 10 nodes by betweenness centrality (sampled graph):")
for node, centrality in top_betweenness:
    print(f"  Node {node}: centrality={centrality:.6f}")

In [ ]:
# PageRank (works well on large graphs)
print("Computing PageRank...")
start_time = time.time()

pagerank = nx.pagerank(G_lcc, alpha=0.85, max_iter=100)

compute_time = time.time() - start_time
print(f"Computed in {compute_time:.2f} seconds")

top_pagerank = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:10]

print("\nTop 10 nodes by PageRank:")
for node, score in top_pagerank:
    print(f"  Node {node}: PageRank={score:.6f}")

### 4.3 Link Prediction

In [ ]:
# Create a smaller sample for link prediction
LINK_PRED_SAMPLE = 5000
sample_nodes_lp = random.sample(nodes_list, min(LINK_PRED_SAMPLE, len(nodes_list)))
G_lp = G_lcc.subgraph(sample_nodes_lp).copy()

print(f"Link prediction sample - Nodes: {G_lp.number_of_nodes():,}, Edges: {G_lp.number_of_edges():,}")

In [ ]:
# Generate non-edges for prediction
print("Generating candidate pairs for link prediction...")

# Get existing edges
existing_edges = set(G_lp.edges())

# Sample non-edges
non_edges = []
lp_nodes = list(G_lp.nodes())
attempts = 0
max_attempts = 100000

while len(non_edges) < 1000 and attempts < max_attempts:
    u, v = random.sample(lp_nodes, 2)
    if (u, v) not in existing_edges and (v, u) not in existing_edges:
        non_edges.append((u, v))
    attempts += 1

print(f"Generated {len(non_edges)} non-edge pairs for prediction")

In [ ]:
# Jaccard coefficient predictions
print("Computing Jaccard coefficient predictions...")
jaccard_preds = list(nx.jaccard_coefficient(G_lp, non_edges))

# Sort by score
jaccard_preds.sort(key=lambda x: x[2], reverse=True)

print("\nTop 10 predicted links (Jaccard coefficient):")
for u, v, score in jaccard_preds[:10]:
    print(f"  ({u}, {v}): score={score:.4f}")

In [ ]:
# Adamic-Adar predictions
print("Computing Adamic-Adar predictions...")
aa_preds = list(nx.adamic_adar_index(G_lp, non_edges))

# Sort by score
aa_preds.sort(key=lambda x: x[2], reverse=True)

print("\nTop 10 predicted links (Adamic-Adar):")
for u, v, score in aa_preds[:10]:
    print(f"  ({u}, {v}): score={score:.4f}")

In [ ]:
# Common neighbors predictions
print("Computing Common Neighbors predictions...")
cn_preds = list(nx.common_neighbor_centrality(G_lp, non_edges))

# Sort by score
cn_preds.sort(key=lambda x: x[2], reverse=True)

print("\nTop 10 predicted links (Common Neighbors):")
for u, v, score in cn_preds[:10]:
    print(f"  ({u}, {v}): score={score:.4f}")

## Step 5: Visualization and Insights

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Create outputs directory
OUTPUTS_DIR = Path('outputs')
OUTPUTS_DIR.mkdir(exist_ok=True)

In [ ]:
# Plot 1: Path Length Distribution
fig, ax = plt.subplots(figsize=(10, 6))

sns.histplot(path_lengths, bins=range(min(path_lengths), max(path_lengths) + 2), 
             kde=True, ax=ax, color='steelblue', edgecolor='white')

ax.axvline(avg_degree, color='red', linestyle='--', linewidth=2, label=f'Mean: {avg_degree:.2f}')
ax.axvline(median_degree, color='orange', linestyle='--', linewidth=2, label=f'Median: {median_degree:.2f}')
ax.axvline(6, color='green', linestyle=':', linewidth=2, label='Six Degrees Theory')

ax.set_xlabel('Degrees of Separation', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Distribution of Degrees of Separation in soc-Pokec Network', fontsize=14)
ax.legend()

plt.tight_layout()
plt.savefig(OUTPUTS_DIR / 'path_lengths_distribution.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {OUTPUTS_DIR / 'path_lengths_distribution.png'}")

In [ ]:
# Plot 2: Degree Distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Linear scale
axes[0].hist(degrees, bins=100, color='steelblue', edgecolor='white', alpha=0.7)
axes[0].set_xlabel('Degree', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Degree Distribution (Linear Scale)', fontsize=14)

# Log-log scale (power law check)
degree_counts = Counter(degrees)
deg_values = list(degree_counts.keys())
deg_freqs = list(degree_counts.values())

axes[1].scatter(deg_values, deg_freqs, alpha=0.5, s=10, color='steelblue')
axes[1].set_xscale('log')
axes[1].set_yscale('log')
axes[1].set_xlabel('Degree (log)', fontsize=12)
axes[1].set_ylabel('Frequency (log)', fontsize=12)
axes[1].set_title('Degree Distribution (Log-Log Scale)', fontsize=14)

plt.tight_layout()
plt.savefig(OUTPUTS_DIR / 'degree_distribution.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {OUTPUTS_DIR / 'degree_distribution.png'}")

In [ ]:
# Plot 3: Community Size Distribution
fig, ax = plt.subplots(figsize=(10, 6))

comm_sizes = list(community_sizes.values())
sns.histplot(comm_sizes, bins=50, ax=ax, color='coral', edgecolor='white')

ax.set_xlabel('Community Size', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title(f'Community Size Distribution (Louvain, {num_communities} communities)', fontsize=14)

plt.tight_layout()
plt.savefig(OUTPUTS_DIR / 'community_size_distribution.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {OUTPUTS_DIR / 'community_size_distribution.png'}")

In [ ]:
# Plot 4: Clustering Coefficient Distribution
fig, ax = plt.subplots(figsize=(10, 6))

cc_values = list(clustering_coeffs.values())
sns.histplot(cc_values, bins=50, ax=ax, color='mediumseagreen', edgecolor='white')

ax.axvline(avg_clustering, color='red', linestyle='--', linewidth=2, label=f'Mean: {avg_clustering:.4f}')

ax.set_xlabel('Clustering Coefficient', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Distribution of Local Clustering Coefficients', fontsize=14)
ax.legend()

plt.tight_layout()
plt.savefig(OUTPUTS_DIR / 'clustering_distribution.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {OUTPUTS_DIR / 'clustering_distribution.png'}")

In [ ]:
# Plot 5: Network Summary Statistics
fig, ax = plt.subplots(figsize=(10, 8))

# Create summary table
summary_data = {
    'Metric': [
        'Total Nodes', 'Total Edges (Directed)', 'Total Edges (Undirected)',
        'Largest Component Size', 'Average Degree', 'Network Density',
        'Average Clustering Coeff.', 'Reciprocity (Directed)',
        'Avg. Degrees of Separation', 'Number of Communities', 'Modularity'
    ],
    'Value': [
        f"{G.number_of_nodes():,}",
        f"{G.number_of_edges():,}",
        f"{G_undirected.number_of_edges():,}",
        f"{len(largest_cc):,}",
        f"{avg_degree_network:.2f}",
        f"{density:.6f}",
        f"{avg_clustering:.4f}",
        f"{reciprocity:.4f}",
        f"{avg_degree:.2f}",
        f"{num_communities}",
        f"{modularity:.4f}"
    ]
}

ax.axis('off')
table = ax.table(
    cellText=[[m, v] for m, v in zip(summary_data['Metric'], summary_data['Value'])],
    colLabels=['Metric', 'Value'],
    cellLoc='left',
    loc='center',
    colWidths=[0.6, 0.3]
)
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1.2, 1.8)

# Style header
for i in range(2):
    table[(0, i)].set_facecolor('#4472C4')
    table[(0, i)].set_text_props(color='white', fontweight='bold')

ax.set_title('soc-Pokec Network Summary Statistics', fontsize=16, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig(OUTPUTS_DIR / 'network_summary.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {OUTPUTS_DIR / 'network_summary.png'}")

In [ ]:
# Interactive visualization with Plotly (small sample)
import plotly.graph_objects as go

# Sample a small subgraph for visualization
VIZ_SAMPLE_SIZE = 1000
viz_nodes = random.sample(nodes_list, min(VIZ_SAMPLE_SIZE, len(nodes_list)))
G_viz = G_lcc.subgraph(viz_nodes).copy()

print(f"Visualization sample - Nodes: {G_viz.number_of_nodes()}, Edges: {G_viz.number_of_edges()}")

In [ ]:
# Compute layout
print("Computing graph layout...")
pos = nx.spring_layout(G_viz, k=0.5, iterations=50, seed=42)

# Get community colors for viz nodes
viz_partition = community_louvain.best_partition(G_viz)
node_colors = [viz_partition.get(node, 0) for node in G_viz.nodes()]

# Create edge traces
edge_x = []
edge_y = []
for edge in G_viz.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.3, color='#888'),
    hoverinfo='none',
    mode='lines'
)

# Create node traces
node_x = [pos[node][0] for node in G_viz.nodes()]
node_y = [pos[node][1] for node in G_viz.nodes()]
node_degrees = [G_viz.degree(node) for node in G_viz.nodes()]

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='Viridis',
        color=node_colors,
        size=[min(5 + d/2, 20) for d in node_degrees],
        colorbar=dict(
            thickness=15,
            title='Community',
            xanchor='left'
        ),
        line_width=0.5
    )
)

# Add hover text
node_text = [f'Node: {node}<br>Degree: {G_viz.degree(node)}<br>Community: {viz_partition.get(node, "N/A")}' 
             for node in G_viz.nodes()]
node_trace.text = node_text

In [ ]:
# Create figure
fig = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        title=f'soc-Pokec Network Sample ({VIZ_SAMPLE_SIZE} nodes)',
        titlefont_size=16,
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        plot_bgcolor='white'
    )
)

fig.write_html(OUTPUTS_DIR / 'network_visualization.html')
fig.show()
print(f"Saved: {OUTPUTS_DIR / 'network_visualization.html'}")

## Step 6: Summary and Conclusions

In [ ]:
# Final summary
print("="*70)
print("SOCIAL NETWORK ANALYSIS: SIX DEGREES OF SEPARATION")
print("soc-Pokec Dataset Analysis Summary")
print("="*70)

print(f"\n📊 DATASET STATISTICS:")
print(f"   • Total nodes: {G.number_of_nodes():,}")
print(f"   • Total edges (directed): {G.number_of_edges():,}")
print(f"   • Total edges (undirected): {G_undirected.number_of_edges():,}")
print(f"   • Largest connected component: {len(largest_cc):,} nodes ({100*len(largest_cc)/G.number_of_nodes():.1f}%)")

print(f"\n🔗 DEGREES OF SEPARATION:")
print(f"   • Average: {avg_degree:.2f}")
print(f"   • Median: {median_degree:.2f}")
print(f"   • Range: {min_degree} - {max_degree}")
print(f"   • Conclusion: {'Confirms' if avg_degree <= 6 else 'Exceeds'} six degrees theory!")

print(f"\n📈 NETWORK PROPERTIES:")
print(f"   • Average node degree: {avg_degree_network:.2f}")
print(f"   • Network density: {density:.6f}")
print(f"   • Average clustering coefficient: {avg_clustering:.4f}")
print(f"   • Reciprocity: {reciprocity:.4f}")

print(f"\n👥 COMMUNITY STRUCTURE:")
print(f"   • Number of communities: {num_communities}")
print(f"   • Modularity: {modularity:.4f}")

print(f"\n⭐ TOP INFLUENTIAL NODES (by PageRank):")
for i, (node, score) in enumerate(top_pagerank[:5], 1):
    print(f"   {i}. Node {node} (score: {score:.6f})")

print(f"\n💾 OUTPUTS SAVED:")
for f in OUTPUTS_DIR.glob('*'):
    print(f"   • {f.name}")

print("\n" + "="*70)

In [ ]:
# Save results to JSON for later use
import json

results = {
    'dataset': {
        'name': 'soc-Pokec',
        'nodes': G.number_of_nodes(),
        'edges_directed': G.number_of_edges(),
        'edges_undirected': G_undirected.number_of_edges(),
        'largest_component_size': len(largest_cc)
    },
    'degrees_of_separation': {
        'samples': NUM_SAMPLES,
        'average': avg_degree,
        'median': median_degree,
        'std': std_degree,
        'min': min_degree,
        'max': max_degree
    },
    'network_metrics': {
        'average_degree': avg_degree_network,
        'density': density,
        'average_clustering': avg_clustering,
        'reciprocity': reciprocity
    },
    'community_detection': {
        'algorithm': 'Louvain',
        'num_communities': num_communities,
        'modularity': modularity
    },
    'top_nodes': {
        'by_pagerank': [(int(n), float(s)) for n, s in top_pagerank[:10]],
        'by_degree': [(int(n), float(s)) for n, s in top_degree[:10]]
    }
}

with open(OUTPUTS_DIR / 'analysis_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print(f"Results saved to {OUTPUTS_DIR / 'analysis_results.json'}")

In [ ]:
print(f"\n✅ Analysis complete!")
print(f"Memory usage: {get_memory_usage():.2f} GB")